In [2]:
#Importing the data
import numpy as np
import pandas as pd
import datetime as dt

prices = pd.read_csv("airbnb_price.csv")
print(prices)

       listing_id        price                 nbhood_full
0            2595  225 dollars          Manhattan, Midtown
1            3831   89 dollars      Brooklyn, Clinton Hill
2            5099  200 dollars      Manhattan, Murray Hill
3            5178   79 dollars   Manhattan, Hell's Kitchen
4            5238  150 dollars        Manhattan, Chinatown
...           ...          ...                         ...
25204    36425863  129 dollars  Manhattan, Upper East Side
25205    36427429   45 dollars            Queens, Flushing
25206    36438336  235 dollars  Staten Island, Great Kills
25207    36442252  100 dollars           Bronx, Mott Haven
25208    36455809   30 dollars          Brooklyn, Bushwick

[25209 rows x 3 columns]


In [3]:
xls = pd.ExcelFile("airbnb_room_type.xlsx")
room_types = xls.parse(0)
print(room_types)

       listing_id                                 description        room_type
0            2595                       Skylit Midtown Castle  Entire home/apt
1            3831             Cozy Entire Floor of Brownstone  Entire home/apt
2            5099   Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3            5178             Large Furnished Room Near B'way     private room
4            5238          Cute & Cozy Lower East Side 1 bdrm  Entire home/apt
...           ...                                         ...              ...
25204    36425863  Lovely Privet Bedroom with Privet Restroom     PRIVATE ROOM
25205    36427429                    No.2 with queen size bed     PRIVATE ROOM
25206    36438336                             Seas The Moment     Private room
25207    36442252               1B-1B apartment near by Metro  Entire home/apt
25208    36455809     Cozy Private Room in Bushwick, Brooklyn     Private room

[25209 rows x 3 columns]


In [13]:
import pandas as pd

reviews = pd.read_csv('airbnb_last_review.tsv', sep='\t')
print(reviews)

       listing_id    host_name   last_review
0            2595     Jennifer   May 21 2019
1            3831  LisaRoxanne  July 05 2019
2            5099        Chris  June 22 2019
3            5178     Shunichi  June 24 2019
4            5238          Ben  June 09 2019
...           ...          ...           ...
25204    36425863        Rusaa  July 07 2019
25205    36427429         H Ai  July 07 2019
25206    36438336          Ben  July 07 2019
25207    36442252       Blaine  July 07 2019
25208    36455809    Christine  July 08 2019

[25209 rows x 3 columns]


In [4]:
#Step 2 :Cleaning the price column

# Remove the currency from the price column
prices['price'] = prices['price'].astype(str)
prices["price"]=prices["price"].str.replace(" dollars", "")

# Convert the price column to a float
prices["price"]=pd.to_numeric(prices["price"])
print(prices["price"])

0        225
1         89
2        200
3         79
4        150
        ... 
25204    129
25205     45
25206    235
25207    100
25208     30
Name: price, Length: 25209, dtype: int64


In [5]:
#step 3: Removing Outliers

# Subset prices for listings costing $0, free_listings
free_listings = prices["price"] == 0

# Update prices by removing all free listings from prices
prices = prices.loc[~free_listings]

# Calculate the average price, avg_price
avg_price = round(prices["price"].mean(), 2)
print(avg_price)

141.82


In [6]:
#step 4:Comparing costs to the private rental market

# Add a new column to the prices DataFrame, price_per_month
prices["price_per_month"] = prices["price"] * 365 / 12

# Calculate average_price_per_month
average_price_per_month = round(prices["price_per_month"].mean(), 2)
print(average_price_per_month)
difference = round((average_price_per_month - 3100),2)
print(difference)

4313.61
1213.61


In [7]:
#step 5: Cleaning the room_type column

# Convert the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()

# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

# Create the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


In [14]:
#  step 6: Working with TimeFrames

# Change the data type of the last_review column to datetime
reviews["last_review"]=pd.to_datetime(reviews["last_review"])

# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()

# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

print(last_reviewed)

2019-07-09


In [18]:
#step 7: Joining the DataFrames

# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = prices.merge(room_types, how="outer", on="listing_id")

# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="listing_id")

# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

print(airbnb_merged)

       listing_id  price                 nbhood_full  price_per_month  \
0            2595  225.0          Manhattan, Midtown      6843.750000   
1            3831   89.0      Brooklyn, Clinton Hill      2707.083333   
2            5099  200.0      Manhattan, Murray Hill      6083.333333   
3            5178   79.0   Manhattan, Hell's Kitchen      2402.916667   
4            5238  150.0        Manhattan, Chinatown      4562.500000   
...           ...    ...                         ...              ...   
25197    36425863  129.0  Manhattan, Upper East Side      3923.750000   
25198    36427429   45.0            Queens, Flushing      1368.750000   
25199    36438336  235.0  Staten Island, Great Kills      7147.916667   
25200    36442252  100.0           Bronx, Mott Haven      3041.666667   
25201    36455809   30.0          Brooklyn, Bushwick       912.500000   

                                      description        room_type  \
0                           Skylit Midtown Castle  en

In [24]:

# Step 8. Analyzing listing prices by NYC borough

# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


In [25]:
# Step 9. Price range by borough

# Create labels for the price range, label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]

# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

# Calculate occurence frequencies for each label, prices_by_borough
prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()

In [26]:
# Step 10. Storing the final result

solution = {'avg_price':avg_price,
            'average_price_per_month': average_price_per_month,  
            'difference':difference,          
            'room_frequencies':room_frequencies, 
            'first_reviewed': first_reviewed,
            'last_reviewed': last_reviewed,
            'prices_by_borough':prices_by_borough}
print(solution)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  

In [29]:
airbnb_merged.to_excel(r'D:\airbnb.xlsx', sheet_name='Your sheet name', index=False)